In [1]:
from pyspark.ml.feature import RFormula
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.regression import LabeledPoint

In [2]:
from pyspark.sql.functions import date_format
sqlc = SQLContext(sc)
callsDf = sqlc.read.load('../../test/calls_11_2020.csv',format="csv", sep=",", inferSchema="true", header="true")
bandDf  = sqlc.read.load('../../test/Bandwidth_traffic_11.csv',format="csv", sep=",", inferSchema="true", header="true")


In [3]:
callsDf.registerTempTable("callsData")
bandDf.registerTempTable("bandData")

In [4]:
callsDf = sqlc.sql("select CUSTOMER_ID,CITY_ID,PBX_ID,LAST_INTERIM_USAGE,LAST_INTERIM_TIME,DAY_HOUR,WEEK_DAY,MONTH_DAY,date_trunc('Minute', CAST(UNIX_TIMESTAMP(UPDATE_DATE, 'dd-MMM-yy HH:mm:ss') AS TIMESTAMP)) as date from callsData")
bandDf = sqlc.sql("select date_trunc('Minute', CAST(UNIX_TIMESTAMP(`Date Time`, 'MM/d/yyyy h:mm:ss a') AS TIMESTAMP) )as date,`Traffic Out (volume)`,`Traffic Out (volume)(RAW)`,`Traffic Out (speed)`,`Traffic Out (speed)(RAW)` from bandData")


In [5]:
callsDf.registerTempTable("calls")
bandDf.registerTempTable("band")

In [6]:
callsBandDf = sqlc.sql("select LAST_INTERIM_USAGE as label, band.date,cast(substring(customer_id,5) as INT) as CUSTOMER_ID,CITY_ID,PBX_ID,LAST_INTERIM_TIME,DAY_HOUR,dayofweek(band.date) as WEEK_DAY,MONTH_DAY,`Traffic Out (volume)(RAW)` as traffic_out_volume,`Traffic Out (speed)(RAW)` as traffic_out_speed from band join calls on band.date = calls.date")

In [151]:
#callsBandDf.createOrReplaceTempView("callsBand")

In [152]:
#sqlc.cacheTable("callsBand")

In [7]:
#callsBandDf.printSchema()

In [8]:
#sqlc.sql("select * from callsBand").show(1)

In [138]:
#formula = RFormula(
#    formula="LAST_INTERIM_USAGE ~ CUSTOMER_ID + CITY_ID + PBX_ID + LAST_INTERIM_TIME + DAY_HOUR + MONTH_DAY + traffic_out_volume +traffic_out_speed",
#    featuresCol="features",
#    labelCol="label")

In [139]:
#output = formula.fit(callsBandDf).transform(callsBandDf)


In [140]:
#output.select("features", "label").show(10,False)

In [141]:
##assembler = VectorAssembler(inputCols=["CUSTOMER_ID" , "CITY_ID" , "PBX_ID" , "LAST_INTERIM_TIME" , "DAY_HOUR" ,"WEEK_DAY" ,"MONTH_DAY", "traffic_out_volume", "traffic_out_speed"],outputCol='features')


In [148]:

##data = assembler.setParams(handleInvalid="skip").transform(callsBandDf).selectExpr ("features","label")
##data = assembler.setParams(handleInvalid="skip").transform(callsBandDf)

In [7]:
#data.printSchema()

In [9]:
parsedData = callsBandDf.rdd.map(lambda row: LabeledPoint(row[0],[row[2:]]))

In [168]:
parsedData.take(1)

[LabeledPoint(2318308.0, [213397.0,4.0,927.0,1638.0,12.0,1.0,1.0,26739559143.0,446477861.7966])]

In [10]:
(trainingData, testData) = parsedData.randomSplit([0.7, 0.3])

In [11]:
model = DecisionTree.trainRegressor(trainingData, categoricalFeaturesInfo={},
                                    impurity='variance', maxDepth=5, maxBins=32)

In [13]:
predictions = model.predict(testData.map(lambda x: x.features))


In [14]:
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)


In [ ]:
testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(testData.count())

In [ ]:
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression tree model:')
print(model.toDebugString())